In [2]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [3]:
#import all libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [4]:
df1 = pd.read_csv('AppleFinalData.csv')
df2 = pd.read_csv('AppleNewsStock.csv')

In [5]:
df1.head(10)

,Date,Open,High,Low,Close,Adj Close,compound,neg,neu,pos
0,2006-12-01,13.1143,13.1900,12.8714,91.32,13.0457,0.7707,0.032,0.905,0.063
1,2006-12-04,13.1257,13.1500,12.9286,91.12,13.0171,0.8720,0.011,0.904,0.085
2,2006-12-05,13.0929,13.1900,12.9814,91.27,13.0386,0.0000,0.000,0.000,0.000
3,2006-12-06,12.9486,13.0557,12.8100,89.83,12.8329,0.6858,0.029,0.878,0.093
4,2006-12-07,12.8614,12.9286,12.4143,87.04,12.4343,-0.6712,0.091,0.869,0.040
5,2006-12-08,12.4614,12.7700,12.4286,88.26,12.6086,-0.1796,0.084,0.848,0.069
6,2006-12-11,12.7000,12.7571,12.5786,88.75,12.6786,-0.8743,0.105,0.852,0.042
7,2006-12-12,12.6586,12.6914,12.2186,86.14,12.3057,0.0000,0.000,0.000,0.000
8,2006-12-13,12.5643,12.7243,12.4500,89.05,12.7214,0.9360,0.018,0.900,0.082
9,2006-12-14,12.7214,12.8571,12.6086,88.55,12.6500,0.9620,0.026,0.880,0.094


In [6]:
df2.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,News
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900,WHAT'S ON TONIGHT : 8 P.M. (TLC) ASHLEY JUDD A...
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200,More on Housing Prices : The broadest governme...
2,2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600,NaN
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100,Honoring R.W. Apple in Words and Food : About ...
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900,"Homebuilders, and Worries Over Jobs, Lead a De..."
5,2006-12-08,12.461429,12.770000,12.428572,88.259995,12.608571,196069300,"Homebuilders, and Worries Over Jobs, Lead a De..."
6,2006-12-11,12.700000,12.757143,12.578571,88.750000,12.678572,124945100,Sales of iPods and iTunes Not Much in Sync : T...
7,2006-12-12,12.658571,12.691428,12.218572,86.139999,12.305715,256655000,NaN
8,2006-12-13,12.564285,12.724286,12.450000,89.050003,12.721429,214263000,Op-Art; iDentities : Op-Art by Seymour Chwast ...
9,2006-12-14,12.721429,12.857142,12.608571,88.549995,12.650000,208082700,"Adam, She’s Ms. Madam : Kristin Chenoweth uses..."
